Python script by [__Hassan Mojeed__](https://linktr.ee/mhola)<br>
Email: mojeed.o.hassan@gmail.com<br>
[Linkedin Profile](https://www.linkedin.com/in/hassanmojeed/)



## View Dashbaord [here](https://app.powerbi.com/view?r=eyJrIjoiOTBiODkwYTYtMjk2MC00MzMzLWJkOGYtZWQzNGFiNTdjODZkIiwidCI6ImFlM2E5OTA2LTc4MWEtNDQ2YS1iZGI2LTYzNzdjMDllMmM2ZiIsImMiOjF9&pageName=ReportSectioneed2047ccf3ea2f7d6e8)

#### Introduction:
The Python code will extract, clean, and load sales data from Jumia Nigeria's sales department into a Microsoft SQL Server database. It will utilize various libraries and handle data processing tasks for sales data from March 2018.

#### Aim:
The objective is to analyze the sales performance of the Jumia Nigeria sales team for the specified period. This will involve extracting  relevant sales data, ensuring consistency and accuracy through cleaning, and finally loading it into an MSSQL Server database for further analysis.

#### Data Source:
Excel files stored in a directory named "Jumia" will serve as the data source. The code will locate these files using the glob function within the specified directory.

#### Data Extraction:
Using pd.read_excel(), the code will read data from the second Excel file found in the "Jumia" directory into a Pandas DataFrame.

#### Data Cleaning and Manipulation:

Column Removal: Unnecessary columns will be dropped from the DataFrame.<br><br>
Column Renaming: Columns will be renamed for clarity and consistency.<br><br>
Handling Missing Values: Missing values in specific columns ('Level', 'Region', and 'sub_category') will be filled with 'Unknown', and conditional replacements will be made based on certain criteria.<br><br>
Value Replacements: Various columns such as 'Region', 'category', 'sub_category', 'brand', and 'item_status' will undergo value replacements to ensure consistency and accuracy.<br><br>
Custom Function Application: The custom function replace_col1_with_col2 will be applied to perform specific replacements in the 'category' column.<br><br>
New Column Creation: A new column named 'final_delivered_date' will be created based on certain conditions, and additional columns will be created to calculate achieved, shipped, and delivered revenues.<br><br>
Data Type Conversion: Certain columns will be converted to appropriate data types.<br><br>
Date Formatting: Date-related columns will be formatted to the desired format.<br>

#### Data Export:
After processing, the DataFrame will be exported to an MSSQL database named 'JumiaDB_Project' and stored in a table named 'Sales_data'. The code will establish a connection to the database using PyODBC and SQLAlchemy, and then export the DataFrame to the specified table. A success message will be printed upon successful export.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
import socket
from glob import glob
from datetime import datetime
import pyodbc
import sqlalchemy
import warnings

# Ignore warnings to enhance readability
warnings.filterwarnings('ignore')

In [2]:
# Get the current working directory and find Excel files in the "Jumia" folder
pwd = os.getcwd() + "/Jumia"
file = glob(pwd + "/*.xlsx")

# Display the list of Excel files found
file

['/Users/mj/Projects/Projects/sales_analysis/Jumia/~$Sales Data.xlsx',
 '/Users/mj/Projects/Projects/sales_analysis/Jumia/Sales Data.xlsx']

In [3]:
# Read data from the second Excel file into a pandas DataFrame
data_import = pd.read_excel(file[1], index_col=None)

In [4]:
# Make a copy of the read DataFrame
data = data_import.copy()


# Display information about the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31864 entries, 0 to 31863
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Level                          31499 non-null  object        
 1   Region                         31484 non-null  object        
 2   year_month                     31864 non-null  object        
 3   year_week                      31864 non-null  object        
 4   date                           31864 non-null  datetime64[ns]
 5   order_nr                       31864 non-null  int64         
 6   payment_method                 31864 non-null  object        
 7   item_status                    31864 non-null  object        
 8   customer_device                31864 non-null  object        
 9   brand                          31861 non-null  object        
 10  category_l2                    31505 non-null  object        
 11  category_l1    

In [5]:
# Check for missing values in the data
data.isna().sum()


Level                              365
Region                             380
year_month                           0
year_week                            0
date                                 0
order_nr                             0
payment_method                       0
item_status                          0
customer_device                      0
brand                                3
category_l2                        359
category_l1                          0
article_name                         0
SKU                                  0
pc1_value_incl_tax                2510
paid_price_incl_tax                  0
coupon_code                      15690
coupon_value_incl_tax                0
selling_price_incl_tax               0
customer_type                        0
id_customer                          0
customer_email                       0
order_placed_by                  31754
Jforce                               0
bob_id_sales_order_item              0
bi_shipped_date          

In [6]:
# Custom Functions

def replace_col1_with_col2(data, column_one, column_two, text_to_replace):
    """
    Replace values in 'column_one' with values in 'column_two'
    where 'column_one' equals 'text_to_replace'.
    
    Parameters:
    - data: DataFrame
    - column_one: str, name of the column to be replaced
    - column_two: str, name of the column with replacement values
    - text_to_replace: str, value to be replaced in 'column_one'
    
    Returns:
    - Modified 'column_one' in the DataFrame
    """
    def switch_one_text(row):
        if row[column_one] == text_to_replace:
            return row[column_two]
        else:
            return row[column_one]
    
    data[column_one] = data.apply(switch_one_text, axis=1)
    return data[column_one].head()

In [7]:
# Columns to be removed from the DataFrame
remove_columns = ['order_placed_by', 'year_month', 'DSC_BI_PRODUCT_CATEGORY_TWO',
                  'DSC_BI_PRODUCT_CATEGORY_THREE', 'mv_nnn', 'year_week']

# Drop specified columns
data.drop(columns=remove_columns, inplace=True)

# Rename columns for clarity
data.rename(columns={'category_l2': 'sub_category', 'category_l1': 'category', 'order_nr': 'order_number'},
            inplace=True)

#data.isna().sum()

In [25]:
# Fill missing values in specific columns with 'Unknown'
data['Level'] = data['Level'].fillna('Unknown')
data['Region'] = data['Region'].fillna('Unknown')
data['sub_category'] = data['sub_category'].fillna('Unknown')

# Conditional replacements based on certain criteria
data['sub_category'] = data.apply(lambda row: 'Mobile Phones' 
                                        if row['category'] == 'Top Searched' 
                                            and row['sub_category'] == 'Unknown'
                                        else row['sub_category'], axis =1)

data['sub_category'] = data.apply(lambda row: 'Mobile Phones' 
                                        if row['brand'] == 'Leagoo'
                                            and row['sub_category'] == 'Unknown'
                                        else row['sub_category'], axis =1)

data['sub_category'] = data.apply(lambda row: 'Peripherals & Accessories' 
                                        if row['brand'] == 'Universal'
                                            and row['sub_category'] == 'Unknown' 
                                            and row['DSC_BI_PRODUCT_CATEGORY_ONE'] == 'Phones'
                                        else row['sub_category'], axis = 1)

data['sub_category'] = data.apply(lambda row: 'Packaging' 
                                        if row['sub_category'] == 'Unknown'
                                            and row['category'] == 'Vendor Packaging'
                                        else row['sub_category'], axis = 1)

data['sub_category'] = data.apply(lambda row: 'Computing' 
                                        if row['sub_category'] == 'Computing'
                                            and row['category'] == 'Clearance Sales'
                                        else row['sub_category'], axis = 1)

data['brand'] = data.apply(lambda row: 'Crown' 
                                        if row['brand'] == 'Crown Micro'
                                            and row['category'] == 'TVs & Electronics'
                                        else row['brand'], axis = 1)

data['brand'] = data.apply(lambda row: 'Garrett' 
                                        if row['brand'] == 'Garret'
                                            and row['category'] == 'TVs & Electronics'
                                        else row['brand'], axis = 1)

# Replacements for specific values in 'Region', 'category' 'sub_category', 'brand' and 'item_status' columns
data['Region'].replace({'SW Region-Lagos' : 'Lagos', 
                        'SS Region': 'South-South', 
                        'SE Region': 'South-East',
                        'SW Region-Others': 'South-West'},
                        inplace=True
                        )

data['category'].replace({'Clearance Sale!': 'Clearance Sales', 
                        'Fashion Outlet': 'Fashion',
                        'Top Searched': 'Phones & Tablets'}, 
                        inplace=True
                        )

data['sub_category'].replace({'Video Games': 'Video Games & Consoles',
                              'All Watches': 'Watches',
                              'Blow Out Sales': 'Sunglasses',
                              'Bridal Shower': 'Brides',
                              'Sony Computer Entertainment': 'Sony',
                              'Clearance Sale GM': 'Clearance Sale',
                              'Computing': 'Laptops'
                              },
                        inplace=True
                        )

data['brand'].replace({'Newage': 'New Age',
                       'Aimegao': 'Aimeigao',
                       'Ameigao': 'Aimeigao',
                       'AMG - AIMEIGAO': 'Aimeigao',
                       'ALBERTO': 'Alberto VO5',
                       'Asantee': 'Asantee Herbal',
                       'Barbara': 'Barbara Fashion',
                       'Brazillian Hair': 'Brazilian Hair',
                       'Cardinal': 'Cardinal Polo Club',
                       'Carossi': 'Carossi Shoe',
                       "Carter's little layette": "Carter's",
                       'DeYoung': 'De Young',
                       'Dynamic': 'Dynamic Collection',
                       'EA': 'EA Sports',
                       'Ecostyler': 'Eco Styler',
                       'eGO CE4': 'EGO',
                       'Frank Olivier': 'Franck Olivier',
                       'Haier': 'Haier Thermocool',
                       'HpPower': 'HP',
                       'HP Power': 'HP',
                       'Indian': 'Indian Hair',
                       'Jacobs': "Jacob's",
                       'Pleasers USA': 'Pleaser USA',
                       'QLT Headsets': 'QLT',
                       'QLT Choice': 'QLT',
                       'SheaMoisture': 'Shea Moisture',
                       'TP Link': 'TP-Link',
                       'Zaron': 'Zaron Cosmetics'
                        },
                        inplace=True
                        )

data['item_status'].replace({'delivered_final': 'delivered'
                             }, 
                        inplace=True
                        )

# Apply custom replacements using the 'replace_col1_with_col2' function
replace_col1_with_col2(data, 
                column_one='category',
                column_two='sub_category', 
                text_to_replace='Shipped From Overseas'
                )


In [10]:
# Create a new column 'final_delivered_date' based on conditions

data['final_delivered_date'] = data.apply(lambda row: row['bi_delivered_date'] 
                                          if pd.isna(row['bi_delivered_final_date']) 
                                          else row['bi_delivered_final_date'], axis=1)

# Achieved revenue: All orders with item status except for "Invalid" and "Fraud"
data['Achieved revenue'] = data.apply(lambda row: np.nan 
                                      if row['item_status'] in ['Invalid', 'Fraud'] 
                                      else row['paid_price_incl_tax'], axis=1).astype(float)

# Shipped revenue: All orders with Bi shipped date
data['Shipped revenue'] = data.apply(lambda row: np.nan 
                                     if pd.isna(row['bi_shipped_date']) 
                                     else row['paid_price_incl_tax'], axis=1).astype(float)

# Delivered revenue: All orders with Item status 'Delivered' and have 'Bi delivered date'.
data['Delivered revenue']  = data.apply(lambda row: row['paid_price_incl_tax']
                                        if row['item_status'] == 'delivered' 
                                            and pd.notna(row['final_delivered_date']) 
                                        else 0, axis =1).astype(float)

# Clean and format column names
columns = data.columns.str.split('_').str.join(' ').str.title()
data.columns = columns.str.split(" ").str.join("_")

In [11]:
# Convert specific columns to the appropriate data types
data['Paid_Price_Incl_Tax']=data['Paid_Price_Incl_Tax'].astype(float)

data['Coupon_Value_Incl_Tax']= data['Coupon_Value_Incl_Tax'].astype(float)

data['Selling_Price_Incl_Tax']=data['Selling_Price_Incl_Tax'].astype(float)


# Extract and format date-related columns
data['Year_Month'] =pd.to_datetime(data['Date']).dt.strftime('%Y-%m')

data['Year_Week'] =pd.to_datetime(data['Date']).dt.strftime('%G-%V')

data['Bi_Delivered_Date']=pd.to_datetime(data['Bi_Delivered_Date']).dt.date

data['Bi_Delivered_Final_Date']=pd.to_datetime(data['Bi_Delivered_Final_Date']).dt.date

data['Bi_Rejected_Date']=pd.to_datetime(data['Bi_Rejected_Date']).dt.date

data['Final_Delivered_Date']=pd.to_datetime(data['Final_Delivered_Date']).dt.date

data['Date']=pd.to_datetime(data['Date']).dt.date

data['Bi_Shipped_Date']=pd.to_datetime(data['Bi_Shipped_Date']).dt.date


In [12]:
# showing the available drivers
pyodbc.drivers()

['SQL Server',
 'MySQL ODBC 8.0 ANSI Driver',
 'MySQL ODBC 8.0 Unicode Driver',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'ODBC Driver 17 for SQL Server']

In [13]:
# Establish a connection to the MSSQL database
database_name = 'JumiaDB_Project'
table_name = 'Sales_data'
socket_name =socket.gethostname()
driver='ODBC Driver 17 for SQL Server'

conn = sqlalchemy.create_engine(f'mssql+pyodbc://{socket_name}/{database_name}?trusted_connection=yes&driver={driver}')

# Export the DataFrame to the MSSQL database
data.to_sql(table_name, con = conn, if_exists='replace', index=False)

# Print a success message
print(f'{table_name} has been successfully exported to {database_name} MSSQL database')


Sales_data has been successfully exported to JumiaDB_Project MSSQL database


## [View exported data](https://drive.google.com/drive/folders/1_TFvO1MgM-I4aWpS2iArAlRys5A26oMg?usp=share_link)